In [0]:
%sh


awk

In [0]:
%python
from pyspark.sql import Row

initial_input = """
Visualização da memória de cálculo

MEMÓRIA DE CÁLCULO (PROCESSAMENTO)

TÍTULO                   NOME                   POSIÇÃO         %PL     GRUPO                                EMISSOR
---------------------------------------------------------------------------------------------------------------
CRI_19K0059589           CRI_19K0059589         4.442.546,23    0,12    10429 - Títulos Privados-Cert.Receb. Imobiliários (CRI)      ASTE - BARI SECURITIZADORA S.A
CRI_19K0059589           CRI_19K0059589         44.499,63       0,00    10429 - Títulos Privados-Cert.Receb. Imobiliários (CRI)      ASTE - BARI SECURITIZADORA S.A
DEBN AEGPA7_092026       DEBN AEGPA7            1445412         7.434,03  0,00    10403 - Títulos Privados-Debêntures simples        AEGP - AEGEA SANEAMENTO PARTICIPAC
DEBN ENGIB1_092029       DEBN ENGIB1            1979541         2.089.971,16 0,06  10403 - Títulos Privados-Debêntures simples        ENGI - ENERGISA SA
DEBN ENGIBG_062028       DEBN ENGIBG            1263178         8.248.768,76 0,23  10403 - Títulos Privados-Debêntures simples        ENGI - ENERGISA SA
DEBN ENGIC6_042027       DEBN ENGIC6            749258          29.526,56   0,00    10403 - Títulos Privados-Debêntures simples        ENGI - ENERGISA SA
"""

dados_tabela = {
    "TITULO": [],
    "NOME": [],
    "POSIÇÃO": [],
    "%PL": [],
    "GRUPO": [],
    "EMISSOR": []
}

linhas = initial_input.strip().split('\n')[6:] #pula as linhas antes dos conteúdos

MAX_ESPACOS_SEGUIDOS = 4 #se temos 3 espaços seguidos, então é uma divisão entre colunas


for linha in linhas:
    espacos_anteriores = 0
    espaco_entre_colunas = False
    valores_coluna = []
    valor_campo = ""
    
    for char in linha:
        if char.isspace():
            espacos_anteriores += 1
            if espacos_anteriores == MAX_ESPACOS_SEGUIDOS:
                espaco_entre_colunas = True
        elif espaco_entre_colunas: #chegamos ao primeiro char depois de um espaço
            valores_coluna.append(valor_campo)
            valor_campo = char
            espacos_anteriores = 0
            espaco_entre_colunas = False
        else: #char normal sem se espaço e sem ter espaço antes
            valor_campo += char

    valores_coluna.append(valor_campo.strip())  # Append the last value after the loop ends
    dados_tabela["TITULO"].append(valores_coluna[0])
    dados_tabela["NOME"].append(valores_coluna[1])
    dados_tabela["POSIÇÃO"].append(valores_coluna[2])
    dados_tabela["%PL"].append(valores_coluna[3])
    dados_tabela["GRUPO"].append(valores_coluna[4])
    dados_tabela["EMISSOR"].append(valores_coluna[5])


for key in dados_tabela:
    print("\n\n")
    print(dados_tabela[key])

#print(dados_tabela)
    

In [0]:
%sql
SELECT MemoriaCalculo FROM desafio_kinea.boletagem_cp.nxenq_memoriacalculo
LIMIT 10

In [0]:
df = _sqldf.toPandas()

texts = list(df["MemoriaCalculo"])

In [0]:

def chega_final_texto(texto_memoria:str)->str:
    index1 = texto_memoria.find("MEMÓRIA DE CÁLCULO (PROCESSAMENTO)") #index do final da string, perto das colunas
    index_coluna = texto_memoria.find("TÍTULO",index1) #index da primeira coluna (TITULO)
    texto_final = texto_memoria[index_coluna:]

    linhas = texto_final.splitlines()
    apenas_linhas_dados = "\n".join(linhas[2:]) #pula linha de coluna e linhas tracejadas
    #print(apenas_linhas_dados)

    return apenas_linhas_dados


textos_processados = list(map(chega_final_texto,texts))

print(textos_processados[3])


prim_linha = textos_processados[3].splitlines()[0]
#print(len(prim_linha))
#print(prim_linha)

def printa_indice_comeco_coluna()->None:
    is_word = False
    for i,char in enumerate(prim_linha):
        if is_word and not char.isspace():
            print(i)
            is_word = False
        if char.isspace():
            is_word = True

In [0]:
from io import StringIO
import pandas as pd

colspecs = [
    (0, 20),   # TÍTULO
    (21, 51),  # NOME
    (52, 72),  # POSIÇÃO
    (73, 83),  # %PL
    (84, 144), # GRUPO
    (144, None) # EMISSOR 
]

df = pd.read_fwf(
    StringIO(textos_processados[4]),
    colspecs=colspecs,
    skiprows=0, 
    header=0
)

df.columns = ["TÍTULO","NOME","POSIÇÃO","%PL","GRUPO","EMISSOR"]

df = df.apply(lambda col: col.str.strip())


#limpa whitespace
df = df.apply(lambda col: col.str.strip())

def para_float_br(x):
    return float(x.replace('.', '').replace(',', '.'))

df["POSIÇÃO"] = df["POSIÇÃO"].apply(para_float_br)
df["%PL"] = df["%PL"].apply(para_float_br)

display(df)